In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from tqdm.notebook import trange, tqdm
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("a")
wandb.login(key=secret_value_0)
wandb.init(project='object_recognition', save_code=True)

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lorenzozanolin-52. Use `wandb login --relogin` to force relogin


## Import everything needed

In [2]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import collections
import math
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Unzip datasets

In [4]:
!pip install py7zr

# WARNING: It can take a lot of time to uncompress!

In [6]:
!python -m py7zr x /kaggle/input/cifar-10/train.7z

In [7]:
!python -m py7zr x /kaggle/input/cifar-10/test.7z

In [8]:
data_dir = '/kaggle/working/'

In [9]:
def read_csv_labels(fname):
    """Read `fname` to return a filename to label dictionary."""
    with open(fname, 'r') as f:
        # Skip the file header line (column name)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

labels = read_csv_labels(os.path.join(data_dir, '/kaggle/input/cifar-10/trainLabels.csv'))
print(f'Number training examples: {len(labels)}')
print(f'Number classes: {len(set(labels.values()))}')

Number training examples: 50000
Number classes: 10


In [10]:
def copyfile(filename, target_dir):
    """Copy a file into a target directory."""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """Split the validation set out of the original training set."""
    # The number of examples of the class that has the fewest examples in the
    # training dataset
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # The number of examples per class for the validation set
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label

def reorg_test(data_dir):
    """Organize the testing set for data loading during prediction."""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))
        
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels('/kaggle/input/cifar-10/trainLabels.csv')
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [11]:
batch_size = 64
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)
wandb.log({'batch_size': batch_size})

### Transformations
We will do some Image Augmentation:

In [15]:
transform_train = torchvision.transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #Random crop can help to make the NN invariant to the position of the element
    transforms.RandomHorizontalFlip(),  #we will use horizontal flip
    torchvision.transforms.ToTensor(), #transform the image into a tensor
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],  #each channel (3) will be normalized using the mean (first tuple) and the std (second tuple)
                                     [0.2023, 0.1994, 0.2010])
])

transform_test = torchvision.transforms.Compose([  #since it is the dataset, we will not do data augmentation, other than normalization
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465], #each channel (3) will be normalized using the mean (first tuple) and the std (second tuple)
                                     [0.2023, 0.1994, 0.2010])]) 

In [16]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds, test_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_test) for folder in ['valid', 'test']]

train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

### ResNet18 Fine Tuning
We will now use transfer learning on a pretrained net (ResNet18) modifying the last layer to predict the classes of the images.

In [17]:
net = torchvision.models.resnet18(pretrained=True)  #load the net with pretrained weigths
net.fc = nn.Linear(net.fc.in_features, 10)  #modify the fc layer to have an output of 10 classes
nn.init.xavier_normal_(net.fc.weight)  #random initialization of the weights 
nn.init.constant_(net.fc.bias, 0);  # bias

device = 'cuda' if torch.cuda.is_available() else 'cpu'  #we want to move the net on the GPU
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net) # if multiple GPUs use them

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 212MB/s]


Now, we will use:
- Cross Entropy as the $Loss function$
- Adam with Reduced LR as $Optimizer$

In [18]:
lr = 1e-4
wandb.log({'optimizer': 'Adam'})
wandb.log({'lr': lr})
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True, min_lr=1e-5, factor=0.5)

Let us train the net!

In [20]:
# we will train for 10 epochs
epochs = 10
wandb.log({'epochs': epochs})

for epoch in trange(epochs):
    mean_train_losses = []
    mean_valid_losses = []
    valid_acc_list = []
    train_losses = []
    valid_losses = []
    total = 0
    val_acc = 0
    # TRAINING
    for X, y in tqdm(train_iter):
        tr_loss = 0
        X = X.to(device)  #move data on the GPU
        y = y.to(device)
        y_pred = net(X)  #calculate the value using the net, y_pred in this case will be an array of 10 values (probability for each class)
        loss = criterion(y_pred, y)  # calculate the loss wrt the ground truth
        
        # zero the gradients before running
        # the backward pass.
        optimizer.zero_grad()

        # Backward pass to compute the gradient
        # of loss w.r.t our learnable params. 
        loss.backward()

        # Update params
        optimizer.step()
        
        train_losses.append(loss.item())
        
    # VALIDATION
    with torch.no_grad():
        net.eval()  # put network in train mode for Dropout and Batch Normalization
        
        for X, y in tqdm(valid_iter):
            X = X.to(device)
            y = y.to(device)
            y_pred = net(X)
            loss = criterion(y_pred, y)
            _, predicted = torch.max(y_pred, 1)
            val_acc += (y == predicted).sum().float()
            valid_losses.append(loss.item())
            total += len(y)

    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    
    accuracy = 100*val_acc/total
    valid_acc_list.append(accuracy)
    
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.2f}%'\
         .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))
    
    wandb.log({'train_loss':mean_train_losses[-1]})
    wandb.log({'val_loss':mean_valid_losses[-1]})
    wandb.log({'val_accuracy':valid_acc_list[-1]})

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/703 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

epoch : 1, train loss : 1.2007, valid loss : 1.1350, valid acc : 71.77%


  0%|          | 0/703 [00:00<?, ?it/s]

...

  0%|          | 0/78 [00:00<?, ?it/s]

epoch : 9, train loss : 0.8548, valid loss : 0.9026, valid acc : 82.31%


  0%|          | 0/703 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

epoch : 10, train loss : 0.8411, valid loss : 0.9117, valid acc : 82.11%


### Submission generation
We will generate the submission.csv file based on the test set

In [29]:
preds = []

net.eval()
with torch.no_grad():
    for X, _ in test_iter:
        X = X.to(device)
        preds.extend(net(X).argmax(dim=1).type(torch.int32).cpu().numpy())
ids = list(range(1, len(test_ds)+1))
ids.sort(key=lambda x: str(x))
df = pd.DataFrame({'id': ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_ds.classes[x])
df.to_csv('submission.csv', index=False)